In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

## Data cleaning

In [ ]:
df = pd.read_csv('/cardio_train.csv', sep=";")
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


In [ ]:
df.isnull().values.any()

False

There are no null values in the dataset

In [ ]:
df.drop(columns=['id'], inplace=True)

In [ ]:
continuous_features = [feature for feature in df.columns if len(df[feature].unique())>25]
print('Continuous Values are : {}'.format(continuous_features))

Continuous Values are : ['age', 'height', 'weight', 'ap_hi', 'ap_lo']


In [ ]:
categorical_features = [feature for feature in df.columns if feature not in continuous_features]

df[categorical_features]=df[categorical_features].astype("category")

In [ ]:
#change categorical features to have right dtype
df.dtypes

age               int64
gender         category
height            int64
weight          float64
ap_hi             int64
ap_lo             int64
cholesterol    category
gluc           category
smoke          category
alco           category
active         category
cardio         category
dtype: object

In [ ]:
print("continuous var: " , continuous_features)
print("categorical var: " , categorical_features)

continuous var:  ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
categorical var:  ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']


In [ ]:
df.describe()

#min height = 55, max height = 250
#min weight = 10, max weight = 200

,age,height,weight,ap_hi,ap_lo
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,19468.865814,164.359229,74.205690,128.817286,96.630414
std,2467.251667,8.210126,14.395757,154.011419,188.472530
min,10798.000000,55.000000,10.000000,-150.000000,-70.000000
25%,17664.000000,159.000000,65.000000,120.000000,80.000000
50%,19703.000000,165.000000,72.000000,120.000000,80.000000
75%,21327.000000,170.000000,82.000000,140.000000,90.000000
max,23713.000000,250.000000,200.000000,16020.000000,11000.000000


In [ ]:

cleaned_df = df.copy()
cleaned_df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [ ]:
#remove outliers above upper fence and below lower fence
for feature in ['height','weight']:
        Q1 = np.percentile(df[feature], 25.) # 25th percentile of the data of the given feature
        Q3 = np.percentile(df[feature], 75.) # 75th percentile of the data of the given feature
        IQR = Q3-Q1 #Interquartile Range
        outlier_step = IQR * 1.5 
        upper_fence = Q3 + outlier_step
        lower_fence = Q1 - outlier_step
        outliers = df[feature][~((df[feature] >= lower_fence) & (df[feature] <= upper_fence))].index.tolist()  
        print('For the feature {}, No of Outliers is {}'.format(feature, len(outliers)))
        cleaned_df = cleaned_df[~((cleaned_df[feature] < lower_fence) | (cleaned_df[feature]> upper_fence))]

For the feature height, No of Outliers is 519
For the feature weight, No of Outliers is 1819


In [ ]:
cleaned_df.describe()

#new min height = 143, new max height = 186
#new min weight = 40, new max weight = 107

,age,height,weight,ap_hi,ap_lo
count,67723.000000,67723.000000,67723.000000,67723.000000,67723.000000
mean,19468.502163,164.310013,73.059601,128.585001,95.993030
std,2468.007310,7.565537,12.407575,156.523767,189.111316
min,10798.000000,143.000000,40.000000,-150.000000,-70.000000
25%,17664.000000,159.000000,65.000000,120.000000,80.000000
50%,19703.000000,165.000000,72.000000,120.000000,80.000000
75%,21324.000000,170.000000,81.000000,140.000000,90.000000
max,23713.000000,186.000000,107.000000,16020.000000,11000.000000


In [ ]:
#remove ap_hi and ap_lo values that are negative
cleaned_df = cleaned_df[cleaned_df['ap_lo']>=0]
cleaned_df = cleaned_df[cleaned_df['ap_hi']>=0]

In [ ]:
#remove rows where ap_hi < ap_lo
print('There are {} observations where ap_hi < ap_lo'.format(len(cleaned_df[cleaned_df['ap_hi'] < cleaned_df['ap_lo']])))
cleaned_df = cleaned_df[cleaned_df['ap_hi'] >= cleaned_df['ap_lo']].reset_index(drop=True)

There are 1143 observations where ap_hi < ap_lo


In [ ]:
#check for rows where ap_hi > 350
(cleaned_df['ap_hi']>350).sum()


39

In [ ]:
#remove rows where ap_hi < 20
(cleaned_df['ap_hi']<20).sum()

3

In [ ]:
cleaned_df= cleaned_df[(cleaned_df['ap_lo']<350) & (cleaned_df['ap_hi']<350)].copy()
cleaned_df= cleaned_df[(cleaned_df['ap_lo']>20) & (cleaned_df['ap_hi']>20)].copy()

In [ ]:
cleaned_df.describe()

,age,height,weight,ap_hi,ap_lo
count,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000
mean,19463.983275,164.312954,72.997406,126.384124,81.166012
std,2468.911974,7.553720,12.382733,16.502193,9.386770
min,10798.000000,143.000000,40.000000,60.000000,30.000000
25%,17657.000000,159.000000,65.000000,120.000000,80.000000
50%,19702.000000,165.000000,71.000000,120.000000,80.000000
75%,21323.000000,170.000000,81.000000,140.000000,90.000000
max,23713.000000,186.000000,107.000000,240.000000,182.000000


In [ ]:
print('Total observations preserved : {}'.format(len(cleaned_df)))

Total observations preserved : 66489


In [ ]:
cleaned_df.head(3)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [ ]:
cleaned_df["BMI"] = round(cleaned_df["weight"] / (cleaned_df["height"]/100)**2,3)
cleaned_df['MAP'] = round(cleaned_df['ap_lo'] + ((cleaned_df['ap_hi']-cleaned_df['ap_lo'])/3),3)

In [ ]:
cleaned_df_nooe=cleaned_df.copy()
y = cleaned_df.cardio
# Only uncomment when you want standardized data
y = y.reset_index(drop=True)
y

0        0
1        1
2        1
3        1
4        0
        ..
66484    1
66485    0
66486    1
66487    1
66488    0
Name: cardio, Length: 66489, dtype: category
Categories (2, int64): [0, 1]

In [ ]:

X_number = cleaned_df.select_dtypes(include = 'number')
X_category = cleaned_df.drop('cardio',axis = 1).select_dtypes(exclude='number')

# Only uncomment when you want standardized data
X_number = (X_number - X_number.mean())/X_number.std()
X_category = pd.get_dummies(X_category)

cleaned_df = pd.concat([X_number,X_category],axis=1)
cleaned_df['cardio'] = y
cleaned_df.head(3)

,age,height,weight,ap_hi,ap_lo,BMI,MAP,gender_1,gender_2,cholesterol_1,...,gluc_1,gluc_2,gluc_3,smoke_0,smoke_1,alco_0,alco_1,active_0,active_1,cardio
0,-0.433788,0.488110,-0.888124,-0.992845,-0.124219,-1.112179,-0.569948,0,1,1,...,1,0,0,1,0,1,0,0,1,0
1,0.309455,-1.100511,0.969301,0.825095,0.941111,1.707131,0.952717,1,0,0,...,1,0,0,1,0,1,0,0,1,1
2,-0.245851,0.090955,-0.726609,0.219115,-1.189548,-0.776977,-0.569948,1,0,0,...,1,0,0,1,0,1,0,1,0,1


In [ ]:
cleaned_df.isnull().values.any()

True

In [ ]:
cleaned_df.describe()

,age,height,weight,ap_hi,ap_lo,BMI,MAP,gender_1,gender_2,cholesterol_1,...,cholesterol_3,gluc_1,gluc_2,gluc_3,smoke_0,smoke_1,alco_0,alco_1,active_0,active_1
count,6.648900e+04,6.648900e+04,6.648900e+04,6.648900e+04,6.648900e+04,6.648900e+04,6.648900e+04,66489.000000,66489.000000,66489.000000,...,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000,66489.000000
mean,3.427734e-17,8.869895e-17,-1.429175e-15,-3.205986e-17,-6.702381e-16,-1.678868e-16,-7.628110e-16,0.654680,0.345320,0.754095,...,0.111988,0.853991,0.071636,0.074373,0.913324,0.086676,0.947615,0.052385,0.195762,0.804238
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.475476,0.475476,0.430626,...,0.315355,0.353118,0.257886,0.262379,0.281362,0.281362,0.222803,0.222803,0.396789,0.396789
min,-3.510041e+00,-2.821518e+00,-2.664792e+00,-4.022746e+00,-5.450865e+00,-2.949374e+00,-4.528768e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.318946e-01,-7.033560e-01,-6.458515e-01,-3.868652e-01,-1.242187e-01,-7.073695e-01,-2.654514e-01,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
50%,9.640551e-02,9.095465e-02,-1.613058e-01,-3.868652e-01,-1.242187e-01,-1.872732e-01,-2.654514e-01,1.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,7.529700e-01,7.528802e-01,6.462704e-01,8.250950e-01,9.411105e-01,5.825391e-01,6.481295e-01,1.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.721008e+00,2.871042e+00,2.745968e+00,6.884896e+00,1.074214e+01,5.179660e+00,8.261365e+00,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
cleaned_df.to_csv("./cleaned_df.csv", index=False)

In [ ]:
cleaned_df_nooe.head(3)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI,MAP
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,21.967,90.000
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,34.928,106.667
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,23.508,90.000


## Feature engineering

In [ ]:
#feat eng on training set
X = cleaned_df.drop(columns='cardio')

poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
Xe_with_category_interactions = poly.fit_transform(X)
cols = poly.get_feature_names_out(X.columns)
Xe_with_category_interactions = pd.DataFrame(Xe_with_category_interactions, columns=cols)
Xe_with_category_interactions.head(3)

,age,height,weight,ap_hi,ap_lo,BMI,MAP,gender_1,gender_2,cholesterol_1,...,smoke_1 alco_0,smoke_1 alco_1,smoke_1 active_0,smoke_1 active_1,alco_0 alco_1,alco_0 active_0,alco_0 active_1,alco_1 active_0,alco_1 active_1,active_0 active_1
0,-0.433788,0.488110,-0.888124,-0.992845,-0.124219,-1.112179,-0.569948,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.309455,-1.100511,0.969301,0.825095,0.941111,1.707131,0.952717,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.245851,0.090955,-0.726609,0.219115,-1.189548,-0.776977,-0.569948,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
Xe_with_category_interactions.drop(columns=['gender_1 gender_2','cholesterol_1 cholesterol_2', 'cholesterol_1 cholesterol_3', 'cholesterol_2 cholesterol_3', 'gluc_1 gluc_2', 'gluc_1 gluc_3', 'gluc_2 gluc_3', 'smoke_0 smoke_1', 'alco_0 alco_1', 'active_0 active_1', 'weight BMI', 'height BMI', 'ap_lo MAP', 'ap_hi MAP'], inplace=True)
Xe_with_category_interactions.head(3)

,age,height,weight,ap_hi,ap_lo,BMI,MAP,gender_1,gender_2,cholesterol_1,...,smoke_0 active_0,smoke_0 active_1,smoke_1 alco_0,smoke_1 alco_1,smoke_1 active_0,smoke_1 active_1,alco_0 active_0,alco_0 active_1,alco_1 active_0,alco_1 active_1
0,-0.433788,0.488110,-0.888124,-0.992845,-0.124219,-1.112179,-0.569948,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.309455,-1.100511,0.969301,0.825095,0.941111,1.707131,0.952717,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.245851,0.090955,-0.726609,0.219115,-1.189548,-0.776977,-0.569948,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
engineered_df_with_category_interactions = pd.DataFrame(Xe_with_category_interactions)
engineered_df_with_category_interactions['cardio'] = y
engineered_df_with_category_interactions.to_csv("./engineered_df_with_category_interactions.csv", index=False)

In [ ]:
engineered_df_with_category_interactions

,age,height,weight,ap_hi,ap_lo,BMI,MAP,gender_1,gender_2,cholesterol_1,...,smoke_0 active_1,smoke_1 alco_0,smoke_1 alco_1,smoke_1 active_0,smoke_1 active_1,alco_0 active_0,alco_0 active_1,alco_1 active_0,alco_1 active_1,cardio
0,-0.433788,0.488110,-0.888124,-0.992845,-0.124219,-1.112179,-0.569948,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,0.309455,-1.100511,0.969301,0.825095,0.941111,1.707131,0.952717,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2,-0.245851,0.090955,-0.726609,0.219115,-1.189548,-0.776977,-0.569948,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
3,-0.745666,0.620495,0.727028,1.431075,2.006440,0.354575,1.866298,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,-0.806016,-1.100511,-1.372670,-1.598825,-2.254877,-0.885085,-2.092613,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66484,0.652116,0.090955,0.565513,1.431075,-0.124219,0.501403,0.648130,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
66485,-0.090721,0.488110,0.242482,-0.386865,-0.124219,-0.033267,-0.265451,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
66486,-0.161198,2.473886,2.584453,3.249015,0.941111,0.929705,2.170795,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
66487,1.201751,-0.173816,-0.080548,0.522105,-0.124219,0.004147,0.191339,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1


#### Feature engineering wirthout categorical interactions for fewer columns

In [ ]:
X_continuous = cleaned_df[continuous_features].copy()
X_continuous['BMI'] = cleaned_df['BMI']
X_continuous['MAP'] = cleaned_df['MAP']
poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
Xe_continuous = poly.fit_transform(X_continuous)
cols = poly.get_feature_names_out(X_continuous.columns)
Xe_continuous = pd.DataFrame(Xe_continuous, columns=cols)
Xe_continuous.head(3)

,age,height,weight,ap_hi,ap_lo,BMI,MAP,age height,age weight,age ap_hi,...,weight ap_hi,weight ap_lo,weight BMI,weight MAP,ap_hi ap_lo,ap_hi BMI,ap_hi MAP,ap_lo BMI,ap_lo MAP,BMI MAP
0,-0.433788,0.488110,-0.888124,-0.992845,-0.124219,-1.112179,-0.569948,-0.211736,0.385257,0.430684,...,0.881770,0.110322,0.987753,0.506185,0.123330,1.104222,0.565870,0.138153,0.070798,0.633884
1,0.309455,-1.100511,0.969301,0.825095,0.941111,1.707131,0.952717,-0.340559,0.299955,0.255330,...,0.799765,0.912219,1.654723,0.923470,0.776506,1.408545,0.786082,1.606599,0.896612,1.626413
2,-0.245851,0.090955,-0.726609,0.219115,-1.189548,-0.776977,-0.569948,-0.022361,0.178637,-0.053870,...,-0.159211,0.864336,0.564558,0.414129,-0.260648,-0.170247,-0.124884,0.924251,0.677980,0.442836


In [ ]:
#Remove superfluous, redundant interactions
Xe_continuous.drop(columns=['height BMI', 'weight BMI', 'ap_hi MAP', 'ap_lo MAP'], inplace=True)
Xe_continuous.columns

Index(['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'BMI', 'MAP', 'age height',
       'age weight', 'age ap_hi', 'age ap_lo', 'age BMI', 'age MAP',
       'height weight', 'height ap_hi', 'height ap_lo', 'height MAP',
       'weight ap_hi', 'weight ap_lo', 'weight MAP', 'ap_hi ap_lo',
       'ap_hi BMI', 'ap_lo BMI', 'BMI MAP'],
      dtype='object')

In [ ]:
engineered_df_without_category_interactions = Xe_continuous.copy()
engineered_df_without_category_interactions[categorical_features] = cleaned_df_nooe[categorical_features].copy()
engineered_df_without_category_interactions.head(3)

,age,height,weight,ap_hi,ap_lo,BMI,MAP,age height,age weight,age ap_hi,...,ap_hi BMI,ap_lo BMI,BMI MAP,gender,cholesterol,gluc,smoke,alco,active,cardio
0,-0.433788,0.488110,-0.888124,-0.992845,-0.124219,-1.112179,-0.569948,-0.211736,0.385257,0.430684,...,1.104222,0.138153,0.633884,2,1,1,0,0,1,0
1,0.309455,-1.100511,0.969301,0.825095,0.941111,1.707131,0.952717,-0.340559,0.299955,0.255330,...,1.408545,1.606599,1.626413,1,3,1,0,0,1,1
2,-0.245851,0.090955,-0.726609,0.219115,-1.189548,-0.776977,-0.569948,-0.022361,0.178637,-0.053870,...,-0.170247,0.924251,0.442836,1,3,1,0,0,0,1


In [ ]:
engineered_df_without_category_interactions.drop(columns='cardio', inplace=True)
engineered_df_without_category_interactions = pd.get_dummies(engineered_df_without_category_interactions)
engineered_df_without_category_interactions['cardio'] = y
engineered_df_without_category_interactions.head(3)

,age,height,weight,ap_hi,ap_lo,BMI,MAP,age height,age weight,age ap_hi,...,gluc_1,gluc_2,gluc_3,smoke_0,smoke_1,alco_0,alco_1,active_0,active_1,cardio
0,-0.433788,0.488110,-0.888124,-0.992845,-0.124219,-1.112179,-0.569948,-0.211736,0.385257,0.430684,...,1,0,0,1,0,1,0,0,1,0
1,0.309455,-1.100511,0.969301,0.825095,0.941111,1.707131,0.952717,-0.340559,0.299955,0.255330,...,1,0,0,1,0,1,0,0,1,1
2,-0.245851,0.090955,-0.726609,0.219115,-1.189548,-0.776977,-0.569948,-0.022361,0.178637,-0.053870,...,1,0,0,1,0,1,0,1,0,1


In [ ]:
engineered_df_without_category_interactions.to_csv('./engineered_df_without_category_interactions.csv', index=False)